In [1]:
import pandas as pd
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn.decomposition import PCA
plt.rcParams['font.sans-serif']=['SimHei'] #图中文字体设置为黑体
plt.rcParams['axes.unicode_minus']=False
from tqdm.notebook import tqdm
from sklearn.metrics import silhouette_score
import scipy
import numpy as np

In [2]:
def read_data(file_name):
    df = pd.read_csv(file_name)
    df = df.drop(labels=['BASIC_entity_name', 'BASIC_year'], axis=1)
    fields_df = pd.read_excel('IDOU_05_事件触发_03_大小类事件映射表_v2.0.xlsx')
    rename_dict = {}
    for i, row in fields_df.iterrows():
        rename_dict[row['大类事件代码'] + '_' + row['小类事件代码']] = row['名称']
    df = df.rename(columns=rename_dict)
    if 'REVENUEAB_test' in df.columns:
        df = df.drop(labels='REVENUEAB_test', axis=1)
    df = df.transpose()
    df.columns = df.columns.astype(str)
    return df

def gen_cluster_dict(df, cluster_object):
    k = max(cluster_object.labels_) + 1
    temp_df = df.copy(deep=True)
    temp_df['cluster'] = cluster_object.labels_
    cluster_dict = {}
    for i in range(k):
        cluster_dict[i] = temp_df[temp_df['cluster'] == i].index.tolist()
    return cluster_dict

def tsne_plot(df, cluster_object):
    tsne = TSNE()
    tsne.fit_transform(df)  # 进行数据降维
    tsne_df = pd.DataFrame(tsne.embedding_, index=df.index)  # 转换数据格式
    tsne_df['cluster'] = cluster_object.labels_
    plt.figure(figsize=(16, 10))
    ax = sns.scatterplot(x=0, y=1, hue="cluster", data=tsne_df)
    for i in range(len(tsne_df)):
        ax.text(tsne_df.iloc[i,0], tsne_df.iloc[i,1], tsne_df.index[i], fontsize=8)

def pca_plot(df, cluster_object):
    pca = PCA(n_components=2)
    pca_df = pca.fit_transform(df)
    print(pca.explained_variance_ratio_)
    pca_df = pd.DataFrame(pca_df, index=df.index, columns=[0, 1])
    pca_df['cluster'] = cluster_object.labels_
    plt.figure(figsize=(16, 10))
    ax = sns.scatterplot(x=0, y=1, hue="cluster", data=pca_df)
    for i in range(len(pca_df)):
        ax.text(pca_df.iloc[i,0], pca_df.iloc[i,1], pca_df.index[i], fontsize=8)

In [3]:
df_all = read_data('舞弊案例全市场数据2014-2021.csv')
df_part = read_data('all_samples.csv')
# df_all.to_csv('全数据-转换后.csv')
# df_part.to_csv('部分数据-转换后.csv')

In [4]:
df = df_all

In [5]:
# sse = {}
# sc = {}
# for k in range(2, 40):
#     kmeans = cluster.KMeans(n_clusters=k, max_iter=1000, n_init=10, random_state=0).fit(df)
#     sse[k] = kmeans.inertia_
#     sc[k] = silhouette_score(df, kmeans.predict(df))
# plt.figure()
# plt.plot(list(sse.keys()), list(sse.values()))
# plt.xlabel("Number of cluster")
# plt.ylabel("SSE")
# plt.show()
# plt.plot(list(sc.keys()), list(sc.values()))
# plt.xlabel("Number of cluster")
# plt.ylabel("silhouette_score")
# plt.show()

In [6]:
k = 8
cluster_object = cluster.KMeans(n_clusters=k, max_iter=1000, n_init=10, random_state=0).fit(df)
gen_cluster_dict(df, cluster_object)

{0: ['资产质量异常-商誉质量存疑', '关联投资异常-商誉质量', '资产质量异常-推高软性资产'],
 1: ['收入增长异常-赊销比例提高', '收入增长异常-现金流与营收背离'],
 2: ['成本费用异常-会计政策调节', '负债结构异常-融资结构变化'],
 3: ['盈余质量异常-收入结构异常',
  '成本费用异常-期间费用调节',
  '筹资活动异常-存贷双高',
  '关联方交易异常-关联方占款风险',
  '投资活动异常-资产处置',
  '资产质量异常-存货异常',
  '收入增长异常-赊销质量',
  '现金质量异常-货币资金受限',
  '成本费用异常-采购现金流不匹配',
  '现金质量异常-存款真实性',
  '关联投资异常-合并利润调节',
  '筹资活动异常-闲置资金',
  '收入增长异常-大幅收入增长',
  '收入增长异常-税金与营收背离',
  '财务困境-自由现金流',
  '资产质量异常-预付账款质量存疑',
  '现金质量异常-经营现金流质量',
  '关联投资异常-承担集团融投资职能',
  '成本费用异常-长期待摊费用异常',
  '负债结构异常-负债结构变化',
  '财务困境-净利润连续两年为负',
  '盈余质量异常-利润结构异常',
  '收入增长异常-成本营收背离',
  '收入增长异常-销售费用率与营收背离',
  '成本费用异常-利润增长超出收入增长',
  '盈余质量异常-应计项目较高',
  '财务困境-经营活动现金净流量',
  '资产质量异常-大额长期应收款',
  '筹资活动异常-高息融资'],
 4: ['关联投资异常-子公司资金支持'],
 5: ['关联方交易异常-关联方利润粉饰风险'],
 6: ['成本费用异常-可操纵应计费用调节'],
 7: ['投资活动异常-建设投资扩张', '投资活动异常-激进投资策略']}

In [7]:
cluster_object = cluster.SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=0).fit(df)
gen_cluster_dict(df, cluster_object)

{0: ['盈余质量异常-收入结构异常', '关联方交易异常-关联方占款风险', '关联方交易异常-关联方利润粉饰风险'],
 1: ['成本费用异常-期间费用调节',
  '收入增长异常-大幅收入增长',
  '成本费用异常-可操纵应计费用调节',
  '成本费用异常-会计政策调节',
  '收入增长异常-销售费用率与营收背离',
  '成本费用异常-利润增长超出收入增长'],
 2: ['资产质量异常-推高软性资产', '关联投资异常-子公司资金支持', '负债结构异常-融资结构变化'],
 3: ['现金质量异常-经营现金流质量', '盈余质量异常-利润结构异常', '盈余质量异常-应计项目较高'],
 4: ['投资活动异常-建设投资扩张', '投资活动异常-激进投资策略'],
 5: ['资产质量异常-商誉质量存疑', '关联投资异常-商誉质量', '关联投资异常-合并利润调节', '资产质量异常-预付账款质量存疑'],
 6: ['筹资活动异常-存贷双高',
  '投资活动异常-资产处置',
  '资产质量异常-存货异常',
  '现金质量异常-货币资金受限',
  '成本费用异常-采购现金流不匹配',
  '现金质量异常-存款真实性',
  '筹资活动异常-闲置资金',
  '财务困境-自由现金流',
  '关联投资异常-承担集团融投资职能',
  '成本费用异常-长期待摊费用异常',
  '负债结构异常-负债结构变化',
  '财务困境-净利润连续两年为负',
  '收入增长异常-成本营收背离',
  '财务困境-经营活动现金净流量',
  '资产质量异常-大额长期应收款',
  '筹资活动异常-高息融资'],
 7: ['收入增长异常-赊销比例提高', '收入增长异常-赊销质量', '收入增长异常-税金与营收背离', '收入增长异常-现金流与营收背离']}

In [8]:
cluster_object = cluster.AffinityPropagation(random_state=0).fit(df)
gen_cluster_dict(df, cluster_object)

{0: ['成本费用异常-采购现金流不匹配'],
 1: ['成本费用异常-可操纵应计费用调节'],
 2: ['收入增长异常-现金流与营收背离'],
 3: ['资产质量异常-推高软性资产'],
 4: ['成本费用异常-会计政策调节'],
 5: ['成本费用异常-利润增长超出收入增长'],
 6: ['关联投资异常-子公司资金支持'],
 7: ['关联方交易异常-关联方利润粉饰风险'],
 8: ['资产质量异常-商誉质量存疑',
  '盈余质量异常-收入结构异常',
  '成本费用异常-期间费用调节',
  '筹资活动异常-存贷双高',
  '收入增长异常-赊销比例提高',
  '关联方交易异常-关联方占款风险',
  '投资活动异常-资产处置',
  '资产质量异常-存货异常',
  '收入增长异常-赊销质量',
  '现金质量异常-货币资金受限',
  '关联投资异常-商誉质量',
  '现金质量异常-存款真实性',
  '关联投资异常-合并利润调节',
  '筹资活动异常-闲置资金',
  '收入增长异常-大幅收入增长',
  '收入增长异常-税金与营收背离',
  '财务困境-自由现金流',
  '资产质量异常-预付账款质量存疑',
  '现金质量异常-经营现金流质量',
  '关联投资异常-承担集团融投资职能',
  '成本费用异常-长期待摊费用异常',
  '负债结构异常-负债结构变化',
  '财务困境-净利润连续两年为负',
  '盈余质量异常-利润结构异常',
  '收入增长异常-成本营收背离',
  '收入增长异常-销售费用率与营收背离',
  '盈余质量异常-应计项目较高',
  '财务困境-经营活动现金净流量',
  '资产质量异常-大额长期应收款',
  '筹资活动异常-高息融资'],
 9: ['投资活动异常-建设投资扩张', '投资活动异常-激进投资策略'],
 10: ['负债结构异常-融资结构变化']}

In [9]:
cluster_object = cluster.AgglomerativeClustering(n_clusters=k).fit(df)
gen_cluster_dict(df, cluster_object)

{0: ['收入增长异常-赊销比例提高',
  '收入增长异常-赊销质量',
  '成本费用异常-采购现金流不匹配',
  '收入增长异常-税金与营收背离',
  '收入增长异常-现金流与营收背离'],
 1: ['资产质量异常-推高软性资产', '负债结构异常-融资结构变化'],
 2: ['成本费用异常-利润增长超出收入增长', '关联方交易异常-关联方利润粉饰风险'],
 3: ['投资活动异常-建设投资扩张', '投资活动异常-激进投资策略'],
 4: ['成本费用异常-会计政策调节'],
 5: ['资产质量异常-商誉质量存疑',
  '盈余质量异常-收入结构异常',
  '成本费用异常-期间费用调节',
  '筹资活动异常-存贷双高',
  '关联方交易异常-关联方占款风险',
  '投资活动异常-资产处置',
  '资产质量异常-存货异常',
  '现金质量异常-货币资金受限',
  '关联投资异常-商誉质量',
  '现金质量异常-存款真实性',
  '关联投资异常-合并利润调节',
  '筹资活动异常-闲置资金',
  '收入增长异常-大幅收入增长',
  '财务困境-自由现金流',
  '资产质量异常-预付账款质量存疑',
  '现金质量异常-经营现金流质量',
  '关联投资异常-承担集团融投资职能',
  '成本费用异常-长期待摊费用异常',
  '负债结构异常-负债结构变化',
  '财务困境-净利润连续两年为负',
  '盈余质量异常-利润结构异常',
  '收入增长异常-成本营收背离',
  '收入增长异常-销售费用率与营收背离',
  '盈余质量异常-应计项目较高',
  '财务困境-经营活动现金净流量',
  '资产质量异常-大额长期应收款',
  '筹资活动异常-高息融资'],
 6: ['成本费用异常-可操纵应计费用调节'],
 7: ['关联投资异常-子公司资金支持']}

In [10]:
cluster_object = cluster.Birch(n_clusters=k).fit(df)
gen_cluster_dict(df, cluster_object)

{0: ['收入增长异常-赊销比例提高',
  '收入增长异常-赊销质量',
  '成本费用异常-采购现金流不匹配',
  '收入增长异常-税金与营收背离',
  '收入增长异常-现金流与营收背离'],
 1: ['资产质量异常-推高软性资产', '负债结构异常-融资结构变化'],
 2: ['成本费用异常-利润增长超出收入增长', '关联方交易异常-关联方利润粉饰风险'],
 3: ['投资活动异常-建设投资扩张', '投资活动异常-激进投资策略'],
 4: ['成本费用异常-会计政策调节'],
 5: ['资产质量异常-商誉质量存疑',
  '盈余质量异常-收入结构异常',
  '成本费用异常-期间费用调节',
  '筹资活动异常-存贷双高',
  '关联方交易异常-关联方占款风险',
  '投资活动异常-资产处置',
  '资产质量异常-存货异常',
  '现金质量异常-货币资金受限',
  '关联投资异常-商誉质量',
  '现金质量异常-存款真实性',
  '关联投资异常-合并利润调节',
  '筹资活动异常-闲置资金',
  '收入增长异常-大幅收入增长',
  '财务困境-自由现金流',
  '资产质量异常-预付账款质量存疑',
  '现金质量异常-经营现金流质量',
  '关联投资异常-承担集团融投资职能',
  '成本费用异常-长期待摊费用异常',
  '负债结构异常-负债结构变化',
  '财务困境-净利润连续两年为负',
  '盈余质量异常-利润结构异常',
  '收入增长异常-成本营收背离',
  '收入增长异常-销售费用率与营收背离',
  '盈余质量异常-应计项目较高',
  '财务困境-经营活动现金净流量',
  '资产质量异常-大额长期应收款',
  '筹资活动异常-高息融资'],
 6: ['成本费用异常-可操纵应计费用调节'],
 7: ['关联投资异常-子公司资金支持']}

In [11]:
cluster_object = cluster.BisectingKMeans(n_clusters=k, random_state=0).fit(df)
gen_cluster_dict(df, cluster_object)

{0: ['投资活动异常-建设投资扩张',
  '成本费用异常-可操纵应计费用调节',
  '资产质量异常-推高软性资产',
  '成本费用异常-会计政策调节',
  '成本费用异常-利润增长超出收入增长',
  '关联投资异常-子公司资金支持',
  '关联方交易异常-关联方利润粉饰风险',
  '投资活动异常-激进投资策略',
  '负债结构异常-融资结构变化'],
 1: ['关联投资异常-商誉质量', '关联投资异常-合并利润调节', '关联投资异常-承担集团融投资职能'],
 2: ['成本费用异常-期间费用调节'],
 3: ['现金质量异常-货币资金受限'],
 4: ['资产质量异常-商誉质量存疑',
  '盈余质量异常-收入结构异常',
  '筹资活动异常-存贷双高',
  '收入增长异常-赊销比例提高',
  '关联方交易异常-关联方占款风险',
  '资产质量异常-存货异常',
  '收入增长异常-赊销质量',
  '成本费用异常-采购现金流不匹配',
  '现金质量异常-存款真实性',
  '筹资活动异常-闲置资金',
  '收入增长异常-税金与营收背离',
  '财务困境-自由现金流',
  '资产质量异常-预付账款质量存疑',
  '现金质量异常-经营现金流质量',
  '收入增长异常-现金流与营收背离',
  '成本费用异常-长期待摊费用异常',
  '负债结构异常-负债结构变化',
  '财务困境-净利润连续两年为负',
  '盈余质量异常-利润结构异常',
  '收入增长异常-成本营收背离',
  '收入增长异常-销售费用率与营收背离',
  '盈余质量异常-应计项目较高',
  '财务困境-经营活动现金净流量',
  '资产质量异常-大额长期应收款'],
 5: ['筹资活动异常-高息融资'],
 6: ['投资活动异常-资产处置'],
 7: ['收入增长异常-大幅收入增长']}

In [12]:
cluster_object = cluster.MeanShift().fit(df)
gen_cluster_dict(df, cluster_object)

{0: ['资产质量异常-商誉质量存疑',
  '盈余质量异常-收入结构异常',
  '成本费用异常-期间费用调节',
  '筹资活动异常-存贷双高',
  '收入增长异常-赊销比例提高',
  '关联方交易异常-关联方占款风险',
  '投资活动异常-资产处置',
  '资产质量异常-存货异常',
  '收入增长异常-赊销质量',
  '现金质量异常-货币资金受限',
  '投资活动异常-建设投资扩张',
  '关联投资异常-商誉质量',
  '现金质量异常-存款真实性',
  '关联投资异常-合并利润调节',
  '筹资活动异常-闲置资金',
  '收入增长异常-大幅收入增长',
  '收入增长异常-税金与营收背离',
  '财务困境-自由现金流',
  '资产质量异常-预付账款质量存疑',
  '现金质量异常-经营现金流质量',
  '关联投资异常-承担集团融投资职能',
  '成本费用异常-长期待摊费用异常',
  '负债结构异常-负债结构变化',
  '财务困境-净利润连续两年为负',
  '盈余质量异常-利润结构异常',
  '收入增长异常-成本营收背离',
  '收入增长异常-销售费用率与营收背离',
  '盈余质量异常-应计项目较高',
  '财务困境-经营活动现金净流量',
  '资产质量异常-大额长期应收款',
  '筹资活动异常-高息融资'],
 1: ['负债结构异常-融资结构变化'],
 2: ['成本费用异常-会计政策调节'],
 3: ['成本费用异常-利润增长超出收入增长'],
 4: ['关联投资异常-子公司资金支持'],
 5: ['成本费用异常-可操纵应计费用调节'],
 6: ['资产质量异常-推高软性资产'],
 7: ['关联方交易异常-关联方利润粉饰风险'],
 8: ['投资活动异常-激进投资策略'],
 9: ['收入增长异常-现金流与营收背离'],
 10: ['成本费用异常-采购现金流不匹配']}